# Hybrid Quantum-Classical Model with PyTorch and Qiskit Tutorial

This notebook shows how to build a hybrid quantum-classical model using PyTorch and Qiskit.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.utils import algorithm_globals
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

algorithm_globals.random_seed = 42

### Generate Random Dataset

In [ ]:
# Select dataset dimension (num_inputs) and size (num_samples)
num_inputs = 2
num_samples = 20

# Generate random input coordinates (X) and binary labels (y)
X = 2 * algorithm_globals.random.random([num_samples, num_inputs]) - 1
y01 = 1 * (np.sum(X, axis=1) >= 0)
y = 2 * y01 - 1

# Convert to torch Tensors
X_ = Tensor(X)
y01_ = Tensor(y01).reshape(len(y)).long()
y_ = Tensor(y).reshape(len(y), 1)

# Plot dataset
for x, y_target in zip(X, y):
    if y_target == 1:
        plt.plot(x[0], x[1], "bo")
    else:
        plt.plot(x[0], x[1], "go")
plt.plot([-1, 1], [1, -1], "--", color="black")
plt.show()

### Setup Quantum Circuit

In [ ]:
feature_map = ZZFeatureMap(num_inputs)
ansatz = RealAmplitudes(num_inputs)

qc = QuantumCircuit(num_inputs)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)
qc.draw(output="mpl", style="clifford")

### Build and Initialize Quantum Neural Network (QNN)

In [ ]:
qnn1 = EstimatorQNN(
    circuit=qc, input_params=feature_map.parameters, weight_params=ansatz.parameters
)

# Setup PyTorch model using QNN with TorchConnector
initial_weights = 0.1 * (2 * algorithm_globals.random.random(qnn1.num_weights) - 1)
model1 = TorchConnector(qnn1, initial_weights=initial_weights)

### Train the Hybrid Model with PyTorch

In [ ]:
optimizer = LBFGS(model1.parameters(), lr=0.01)
loss_func = MSELoss()

epochs = 20
loss_list = []

# Training loop
for epoch in range(epochs):
    def closure():
        optimizer.zero_grad()
        output = model1(X_)
        loss = loss_func(output, y_)
        loss.backward()
        return loss

    optimizer.step(closure)
    total_loss = closure().item()
    loss_list.append(total_loss)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}")

### Plot Loss Convergence

In [ ]:
plt.plot(loss_list)
plt.title("Hybrid NN Training Convergence")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

### Save and Load the Trained Model

In [ ]:
# Save model
torch.save(model1.state_dict(), "model1.pt")

# Load model
model1.load_state_dict(torch.load("model1.pt"))
model1.eval()

### Evaluate the Model on Test Data

In [ ]:
model1.eval()
with torch.no_grad():
    correct = 0
    for data, target in test_loader:
        output = model1(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

    print(f"Accuracy: {correct / len(test_loader.dataset):.4f}")

In [ ]:
# Check Qiskit Version
import tutorial_magics

%qiskit_version_table
%qiskit_copyright